# Classification Model With Keras


Bu çalışmada Keras kullanarak bir sınıflandırma yapacağız.

Data klasörü içine bakarak kullanacağımız görsellere bakabilirsin.

Toplamda 500 adet görsel mevcuttur.


200 uçak ve 200 araba görseli modeli eğitmek için 50 uçak ve 50 araba görseli modeli test etmek için kullanılacak.


In [3]:
# keras versiyonuna göre yorum bu altaki iki importtan birini yapman gerekebilir
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# keras versiyonuna göre yorum bu yukardaki iki importtan birini yapman 
# yorum satırını değiştirmeyi dene hata alırsan

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224


görüntüler 224 pixele 224 pixel. Sınıflandırma için süper hd bir görüntüye gerek yok. Görüntülerde hızlıca çalışmak sınıflandırma için daha verimli olacak.

değişkenlerimize bilgileri atıyoruz.

In [4]:
train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16


Epoch ve Batch_size önemli kavramlar  

Epoch basitçe veri seti üzerinde kça iterasyon yapılacağını ifade eder

Batch size ise bir seferde incelenecek örnek sayısını ifade eder.


In [5]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)


Burada amaç veri formatının düzgün aktarılmasından emin olmak. Channels Rgb kanalları, eğer rgb kanalları ilk yazılır ise önce kanal sonra boyut verilir. Aksi halde önce boyut sonra kanal ifade ediliyor.

In [6]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


/home/onuralp/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-07-16 15:30:27.249532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 15:30:27.251303: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipp

Burada aslında ağ mimarisini yazdık. daha ileri seviye çalışmalar için mevcut başarılı mimariler kullanmak güzeldir ama ne olduğu görmek için önemli bi adım    



Conv2D , görüntüyü birden fazla görüntüye evriştiren katmandır. 


Aktivasyon , aktivasyon fonksiyonudur. 
MaxPooling2D, verilen boyut matrisindeki değeri maksimuma çıkarmak için kullanılır ve aynısı sonraki 2 katman için kullanılır. daha sonra Düzleştir , evriştirildikten sonra elde edilen görüntünün boyutlarını düzleştirmek için kullanılır. 
Yoğun, bunu tamamen bağlantılı bir model haline getirmek için kullanılır ve gizli katmandır. 
Bırakma, veri kümesine aşırı sığmayı önlemek için kullanılır. 
Yoğun, çıktı katmanının görüntünün hangi kategoriye ait olduğuna karar veren yalnızca bir nöron içermesidir.

In [7]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])


Burada kayıp, optimize ediciler ve metriklerin kullanımını içeren derleme işlevi kullanılmıştır. Burada kullanılan kayıp fonksiyonu Binary_crossentropy, kullanılan optimizer ise rmsprop'tur .

In [11]:
# eğer aşağıda pil ile ilgili hata alaırsan bunu kullan 
!pip3 install pillow

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 1.7 MB/s eta 0:00:0000:0100:010m


In [17]:
import sys
from PIL import Image
sys.modules['Image'] = Image 
from PIL import Image
print(Image.__file__)
import Image
print(Image.__file__)


/home/onuralp/.local/lib/python3.10/site-packages/PIL/Image.py
/home/onuralp/.local/lib/python3.10/site-packages/PIL/Image.py


In [19]:



train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


Found 400 images belonging to 2 classes.


Found 100 images belonging to 2 classes.


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

Şimdi dataGenerator'ın kısmı devreye giriyor. Hangisinde kullandık: 

Görüntüyü yeniden ölçeklendiren, belirli bir aralıkta kesme uygulayan, görüntüyü yakınlaştıran ve görüntüyle yatay çevirme yapan ImageDataGenerator . Bu ImageDataGenerator görüntünün tüm olası yönelimlerini içerir. 
train_datagen.flow_from_directory, train_dataset dizininden veri hazırlamak için kullanılan işlevdir Target_size, görüntünün hedef boyutunu belirtir. 
test_datagen.flow_from_directory , model için test verilerini hazırlamak için kullanılır ve hepsi yukarıdakine benzer. 
fit_generator, verileri yukarıda yapılan modele sığdırmak için kullanılır; kullanılan diğer faktörler,step_per_epochs'tur ve bize modelin eğitim verileri için kaç kez yürütüleceğini bildirir. 
Epochs bize modelin ileri ve geri geçişte kaç kez eğitileceğini söyler. 
validation_data, doğrulama/test verilerini modele beslemek için kullanılır. 
validation_steps doğrulama/test örneklerinin sayısını belirtir.

In [9]:
model.save_weights('model_saved.weights.h5')


ValueError: The filename must end in `.weights.h5`. Received: filepath=model_saved.h5

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

from keras.models import load_model

model = load_model('model_saved.weights.h5')

image = load_img('v_data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])
